In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import classification_report
import numpy as np
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.utils import to_categorical
import joblib  # For saving scaler

# Load the dataset (replace with your file path)
df = pd.read_csv('/kaggle/input/attention-detection-sys/attention_detection_dataset_v1.csv')


# Show the cleaned dataset columns and first few rows
print(df.columns)
print(df.head())

# Step 1: Data Cleaning
# Drop rows or columns with missing values or handle them as needed
df.dropna(inplace=True)

# Define the fixed mapping for the 'pose' column (target variable)
label_mapping = {'forward': 0, 'down': 1, 'left': 2, 'right': 3}

# Step 2: Apply the label mapping to the "pose" column
df["pose"] = df["pose"].map(label_mapping)

# Check if any values are NaN after the mapping (you can drop or handle them as needed)
#df.dropna(subset=["pose"], inplace=True)

# Step 3: Separate features (X) and the target variable (y)
# Assuming the last column is the target variable (pose)
X = df.iloc[:, :-1]  # All columns except the last one (features)
y = df["label"]  


# Step 5: Normalize the data
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# Save the scaler for future use during inference
joblib.dump(scaler, '/kaggle/working/scaler.pkl')

# Step 6: Split the data into training and test sets
X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.2, random_state=42)
num_classes=2

# Step 7: Build a Neural Network Model
model = Sequential()
model.add(Dense(128, activation='relu', input_shape=(X_train.shape[1],)))  # Input layer
model.add(Dense(64, activation='relu'))  # Hidden layer 1
model.add(Dense(32, activation='relu'))  # Hidden layer 2
model.add(Dense(num_classes, activation='softmax'))  # Output layer (for multi-class classification)

# Step 8: Compile the Model
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# One-hot encode the labels
y_train = to_categorical(y_train, num_classes=num_classes)
y_test = to_categorical(y_test, num_classes=num_classes)


# Step 9: Train the Model
history = model.fit(X_train, y_train, epochs=50, batch_size=32, validation_split=0.2, verbose=2)

# Step 10: Evaluate the model on test data
test_loss, test_accuracy = model.evaluate(X_test, y_test, verbose=2)
print(f"Test accuracy: {test_accuracy}")

# Step 11: Make predictions and evaluate the model using classification_report
y_pred = model.predict(X_test)
y_pred_classes = np.argmax(y_pred, axis=1)
y_test_classes = np.argmax(y_test, axis=1)

# Print classification report
print(classification_report(y_test_classes, y_pred_classes))

# Step 12: Save model weights to a file in Kaggle's working directory
weights_file = '/kaggle/working/model3.weights.h5'
model.save_weights(weights_file)
print(f"Model weights saved to '{weights_file}'")


In [ ]:
import numpy as np
import joblib  # To load the scaler and model
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense

# Define the fixed mapping for categorical variables (same as during training)
label_mapping = {'forward': 0, 'down': 1, 'left': 2, 'right': 3}

# Load the pre-trained model weights and scaler
scaler = joblib.load('/kaggle/working/scaler.pkl')
weights_file = '/kaggle/working/model3.weights.h5'

# Step 1: Define the same model architecture as during training
# Ensure the input shape matches the shape used during training
model = Sequential()
model.add(Dense(128, activation='relu', input_shape=(16,)))  # Input layer (adjust 10 to match number of features)
model.add(Dense(64, activation='relu'))  # Hidden layer 1
model.add(Dense(32, activation='relu'))  # Hidden layer 2
model.add(Dense(2, activation='softmax'))  # Output layer with 4 classes (for 'forward', 'down', 'left', 'right')

# Load the trained model weights
model.load_weights(weights_file)

# Step 2: Example input data (replace this with the actual input data you want to predict)
# In the example, the categorical "down" at index 7 needs to be mapped
sample_input = [1,252.9670143,137.8716588,163.1315422,163.1269741,86.70405746,0,"down",-15,10,0,0,0,0,0,0]

# Step 3: Apply the label mapping to the categorical column (index 7)
sample_input[7] = label_mapping[sample_input[7]]  # 'down' -> 1

# Step 4: Preprocess the input data
# Reshape the input sample to be 2D (1 sample, n features) and scale it
sample_input_scaled = scaler.transform(np.array(sample_input).reshape(1, -1))  # Scaling the input

# Step 5: Make predictions
predictions = model.predict(sample_input_scaled)
predicted_class = np.argmax(predictions, axis=1)  # Get the class with the highest probability

# # Step 6: Convert the predicted class back to the corresponding label
# predicted_label = list(label_mapping.keys())[list(label_mapping.values()).index(predicted_class[0])]

# Step 7: Print the predicted class
print(f"Predicted class: {predicted_class}")
